In [1]:
test_input = """Blueprint 1: Each ore robot costs 4 ore. Each clay robot costs 2 ore. Each obsidian robot costs 3 ore and 14 clay. Each geode robot costs 2 ore and 7 obsidian.
Blueprint 2: Each ore robot costs 2 ore. Each clay robot costs 3 ore. Each obsidian robot costs 3 ore and 8 clay. Each geode robot costs 3 ore and 12 obsidian."""

In [2]:
import re
import numpy as np
from functools import cache, lru_cache
import math

In [3]:
reg = ".*Each ore robot costs (\d+) ore\. Each clay robot costs (\d+) ore\. Each obsidian robot costs (\d+) ore and (\d+) clay\. Each geode robot costs (\d+) ore and (\d+) obsidian\."

In [4]:
def add(a, b):
    return tuple(map(lambda x, y: x + y, a, b))

def sub(a, b):
    return tuple(map(lambda x, y: x - y, a, b))

In [5]:
blueprints = []

# for l in test_input.split("\n"):
for l in open("inputs/19").read().split("\n"):
    m = re.match(reg, l)
    costs = list(map(int, m.groups()))

    # ore cost, clay cost, obsidian cost, geode cost
    ore_robot_costs = (costs[0], 0, 0, 0)
    clay_robot_costs = (costs[1], 0, 0, 0)
    obsidian_robot_costs = (costs[2], costs[3], 0, 0)
    geode_robot_costs = (costs[4], 0, costs[5], 0)

    blueprint = (ore_robot_costs, clay_robot_costs, obsidian_robot_costs, geode_robot_costs)
    blueprints.append(blueprint)

In [6]:
blueprints[:3]

[((3, 0, 0, 0), (3, 0, 0, 0), (2, 15, 0, 0), (3, 0, 9, 0)),
 ((4, 0, 0, 0), (4, 0, 0, 0), (4, 5, 0, 0), (3, 0, 7, 0)),
 ((4, 0, 0, 0), (4, 0, 0, 0), (2, 11, 0, 0), (2, 0, 7, 0))]

In [7]:
TIME_LIMIT = 24

In [8]:
START_ORE = (0, 0, 0, 0)
START_ROBOTS = (1, 0, 0, 0)

In [9]:
def upper_bound(time, blueprint, ore_resources, robot_resources):
    # geode robots are gated by obsidian.
    # get a bound on obsidian robots and use that to estimate geode robots

    # assume we're building an obsidian robot every turn
    # of course we can't build both obsidian robots and geode robots. but let's pretend and see
    current_obsidian_robots = robot_resources[-2]
    current_geode_robots = robot_resources[-1]

    current_obsidian = ore_resources[-2]
    current_geode = ore_resources[-1]

    total = current_geode

    required_obsidian = blueprint[-1][-2]

    for t in range(time):
        total += current_geode_robots

        current_obsidian += current_obsidian_robots
        current_obsidian_robots += 1

        # only check obsidian feasibility for building
        if current_obsidian >= required_obsidian:
            current_obsidian -= required_obsidian
            current_geode_robots += 1

    return total

def lower_bound(time, blueprint, ore_resources, robot_resources):
    # current number plus how many they get from their current geode robots till the end
    return ore_resources[-1] + (time-1) * robot_resources[-1]

In [10]:
def total_geodes(blueprint, time):
    current_best_lb = 0

    @lru_cache(maxsize=10_000_000)
    def f(time, blueprint, ore_resources, robot_resources):
        nonlocal current_best_lb

        if time == 0:
            return ore_resources[-1]
        
        ub = upper_bound(time, blueprint, ore_resources, robot_resources)
        lb = lower_bound(time, blueprint, ore_resources, robot_resources)

        if ub < current_best_lb:
            return -1
        
        if lb > current_best_lb:
            current_best_lb = lb

        after_building_resources = [sub(ore_resources, b) for b in blueprint]
        building_feasibilities = [all(a >= 0 for a in r) for r in after_building_resources]

        all_feasible = all(building_feasibilities)

        new_ore_resources = add(ore_resources, robot_resources)

        # don't build. just collect
        # NEVER do this if it's possible to build everything. why would you?
        if not all_feasible:
            max_geodes = f(time-1, blueprint, new_ore_resources, robot_resources)
        else:
            max_geodes = 0

        for i, (feasible, after_building) in enumerate(zip(building_feasibilities, after_building_resources)):
            if not feasible:
                continue

            new_robots = list(robot_resources)
            new_robots[i] += 1
            new_robots = tuple(new_robots)

            new_ore_resources = add(after_building, robot_resources)

            m = f(time-1, blueprint, new_ore_resources, new_robots)
            
            if m > max_geodes:
                max_geodes = m
        
        if max_geodes > current_best_lb:
            current_best_lb = max_geodes

        return max_geodes
    
    return f(time, blueprint, START_ORE, START_ROBOTS)

In [11]:
s = 0

for (i, b) in enumerate(blueprints):
    print(i)
    r = total_geodes(b, TIME_LIMIT)
    print(r)
    s += (i+1) * r
    print()

0
4

1
9

2
3

3
0

4
0

5
0

6
1

7
1

8
1

9
0

10
6

11
5

12
8

13
0

14
1

15
0

16
0

17
0

18
13

19
0

20
5

21
5

22
2

23
5

24
14

25
9

26
4

27
9

28
0

29
3



In [12]:
s

1962

In [13]:
prod = 1

for (i, b) in enumerate(blueprints[:3]):
    print(i)

    r = total_geodes(b, 32)
    print(r)
    prod *= r

0
40
1
58
2
38


In [14]:
prod

88160